## Detrended Autocorrelation Periodicity Scoring (DAPS)
based on http://www.ctralie.com/Research/p53periodicity/moosmullertralie2020_p53periodicity.pdf

In [1]:
import numpy as np
import gudhi as gd
from gudhi.point_cloud.timedelay import TimeDelayEmbedding
from gudhi.hera import wasserstein_distance
from gudhi.representations import PersistenceImage, BettiCurve
from sklearn.decomposition import PCA
import umap
from gudhi.representations.metrics import WassersteinDistance
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.manifold import MDS
import os
from scipy import stats
#%matplotlib widget

In [2]:
datapath =".\\data\\timeseries\\good2"
filelist = os.listdir(datapath)
print(sorted(filelist))
good_labels  =sorted(filelist)
good_time_series = [np.loadtxt(os.path.join(datapath,f)) for f in sorted(filelist)]
datapath =".\\data\\timeseries\\bad2"
filelist = os.listdir(datapath)
print(sorted(filelist))
bad_labels = sorted(filelist)
bad_time_series = [np.loadtxt(os.path.join(datapath,f)) for f in sorted(filelist)]

print(good_time_series[0])

['signal05_good.txt', 'signal10_good.txt', 'signal15_good.txt', 'signal20_good.txt', 'signal25_good.txt']
['signal05_bad.txt', 'signal10_bad.txt', 'signal15_bad.txt', 'signal20_bad.txt', 'signal25_bad.txt']
[ 0.2504463   0.07998543 -0.35629045 ... -0.55235447 -0.22336272
 -0.37701057]


In [27]:
dim = 1041
delay = 1
skip = 1
N=len(good_time_series[0])
M=dim

tde = TimeDelayEmbedding(dim = dim, delay=delay, skip=skip)
good_point_clouds = tde.transform(good_time_series)
bad_point_clouds = tde.transform(bad_time_series)
for i in range(0,len(good_point_clouds)):
    good_point_clouds[i] = good_point_clouds[i]-np.mean(good_point_clouds[i],1)[:, None]
    good_point_clouds[i] = good_point_clouds[i]/np.sqrt(np.sum(good_point_clouds[i]**2, 1))[:, None]
    bad_point_clouds[i] = bad_point_clouds[i]-np.mean(bad_point_clouds[i],1)[:, None]
    bad_point_clouds[i] = bad_point_clouds[i]/np.sqrt(np.sum(good_point_clouds[i]**2, 1))[:, None]

In [28]:
Ys = []

for Xhat in good_point_clouds+bad_point_clouds:
    Y = np.zeros_like(Xhat)
    for i in range(0,len(Xhat)):
        for j in range(0,len(Xhat[i])):
            ar1=np.array(range(1-i,dim))
            ar2 = np.array(range(-N+M-1+j,j-1))
            Y[i][j] =np.mean([Xhat[i+k][j-k] for k in np.intersect1d(ar1,ar2)])

C:\Users\Niklas\AppData\Roaming\Python\Python37\site-packages\numpy\core\fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Niklas\AppData\Roaming\Python\Python37\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


IndexError: index 1041 is out of bounds for axis 0 with size 1041